## Pages List (Buy)

In [561]:
import requests
import re
from bs4 import BeautifulSoup

pages_list = []
main_page_url = 'https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|1||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||'
page = requests.get(main_page_url)
soup = BeautifulSoup(page.text, 'html.parser')
pattern = re.compile("var totalPages = [\d]*;")
total_pages = ''
for script in soup.find_all("script", type="text/javascript"):
    if(pattern.findall(script.text)):
        total_pages = int((pattern.findall(script.text)[0].split()[-1])[0:-1])
for i in range(1, total_pages + 1):
    pages_list.append('https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|'+ str(i) +'||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||')

## Properties Pages List (Buy)

In [562]:
paginator_url = 'https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|1||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||'
page = requests.get(paginator_url)
soup = BeautifulSoup(page.content, 'html.parser')
properties_number = soup.find(id='lblNumInm').get_text()
division = soup.find(id='divAdverts')
properties_pages_list = []
for i in range(0, 34):
    properties = division.find(id='rowIndex_' + str(i))
    property = ''
    if (i == 0):
        property = properties.find('li', class_='media')
        property = property.find('span').attrs.get('onclick')
    else:
        property = properties.find('li', class_='media').attrs.get('onclick')
    url_property = str(property).replace('javascript:window.location=', '')
    url_property = 'https://www.fincaraiz.com.co' + url_property.replace("'", "")
    properties_pages_list.append(url_property)

## Convert String to JSON

In [563]:
import json
paginator_url = 'https://www.fincaraiz.com.co/palermo/medellin/proyecto-nuevo-det-2393445.aspx'
page = requests.get(paginator_url)
soup = BeautifulSoup(page.content, 'html.parser')

pattern = re.compile("var sfAdvert = \{.*\:.*\:.*\};")
json_property = ''
for script in soup.find_all("script", type="text/javascript"):
    if(pattern.findall(script.text)):
        json_property = pattern.findall(script.text)[0].split()[3:]
        json_to_strip = (json_property[-1])[0:-1]
        json_property = json_property[0:-1]
        json_property.append(json_to_strip)
        json_property = " ".join(json_property)

json_finca_raiz = json.loads(json_property)
print (json.dumps(json_finca_raiz, indent = 4))

{
    "AdvertId": "2393445",
    "AdvertWASId": "2393445",
    "AdvertExternalId": "Palermo",
    "ClientId": "350013",
    "ClientName": "Constructora Editec",
    "ClientURL": "constructora-constructora-editec-350013.aspx",
    "AdvertCategoryId": "2",
    "AdvertType": "Used",
    "TransactionId": "1",
    "TransactionType": "Venta",
    "Category1Id": "1",
    "Category2Id": "0",
    "Category3Id": "0",
    "Category1": "Proyecto Nuevo",
    "Category2": "",
    "Category3": "",
    "OriginId": "23",
    "Location1Id": "55",
    "Location2Id": "5500006",
    "Location3Id": "5500102",
    "Location4Id": "5502023",
    "Location1": "Antioquia",
    "Location2": "Medell\u00edn",
    "Location3": "SurOriente",
    "Location4": "Loma del Indio",
    "Title": "Palermo",
    "Description": "PALERMO, donde las cosas importantes se encuentran muy cerca. Proyecto de apartamentos muy exclusivos, contiguo a la Iglesia SAN JOSE DE NAZARETH. Ac\u00e1 encontrar\u00e1 todo a la mano porque en nues

## Property Scraping (Old Properties)

In [564]:
paginator_url = 'https://www.fincaraiz.com.co/apartamento-en-venta/medellin/america-det-4950356.aspx'
page = requests.get(paginator_url)
soup = BeautifulSoup(page.content, 'html.parser')

# Extract Detail Header
detail_header = soup.find('div', class_='row detailheader')
property_price = detail_header.find('div', class_='price').find('h2').get_text()
property_neighborhood = detail_header.find('div', class_='box').find('h1').find('span').text.split()[1:]
property_neighborhood = " ".join(property_neighborhood)

# Extract Location
property_latitude = json_finca_raiz["Latitude"]
property_longitude = json_finca_raiz["Longitude"]

property_array_location = []
location_object = {
    'latitude': property_latitude,
    'longitude': property_longitude
}
property_array_location.append(location_object)

# Extract Property Agency
contact_form = soup.find('div', class_='contact-form')
property_agency = contact_form.find('input', id='hd_UserName').attrs.get('value')

# Extract Small Features
small_features = soup.find('div', class_='features clearfix')
property_sqrt_meters = small_features.find('span', class_='advertSurface').get_text().split()
property_sqrt_meters = float(" ".join(property_sqrt_meters)[0:-3].replace(",", "."))
property_rooms = small_features.find('span', class_='advertRooms').get_text().split()
property_rooms = int(" ".join(property_rooms)[-1])
property_baths = small_features.find('span', class_='advertBaths').get_text().split()
property_baths = int(" ".join(property_baths)[-1])
property_park = small_features.find('span', class_='advertGarages').get_text().split()
property_park = property_park[-1]
if(property_park == 'especificar'):
    property_park = int('0')
else:
    property_park = int(property_park)

# Estract Big Features
big_features = soup.find('ul', class_='boxcube')
property_private_area = ''
property_const_area = ''
property_sqrt_meters_price = ''
property_stratum = ''
property_feature_status = ''
property_antiquity = ''
property_floor = ''
property_sector = ''
property_weather = ''
property_admon = ''
property_apartment_type = ''

for li in big_features.find_all('li'):
    value = li.get_text().split()
    value = " ".join(value)
    if(value.find('Área privada') == 0):
        property_private_area = value.replace('Área privada: ', '')[0:-3]
    elif(value.find('Área Const') == 0):
        property_const_area = value.replace('Área Const.: ', '')[0:-3]
    elif(value.find('Precio m²') == 0):
        property_sqrt_meters_price = value.replace('Precio m²: ', '')[0:-3]
    elif(value.find('Estrato') == 0):
        property_stratum = int(value.replace('Estrato: ', '')[0])
    elif(value.find('Estado') == 0):
        property_feature_status = value.replace('Estado: ', '')
    elif(value.find('Antigüedad') == 0):
        property_antiquity = value.replace('Antigüedad: ', '')
    elif(value.find('Piso No') == 0):
        property_floor = value.replace('Piso No: ', '')[0:-1]
    elif(value.find('Sector') == 0):
        property_sector = value.replace('Sector: ', '')
    elif(value.find('Clima') == 0):
        property_weather = value.replace('Clima: ', '')
    elif(value.find('Admón') == 0):
        property_admon = value.replace('Admón: ', '').split()[0]
    elif(value.find('Tipo de Apartamento') == 0):
        property_apartment_type = value.replace('Tipo de Apartamento: ', '')

property_features = []
features_object = {
    'squareMeters': property_sqrt_meters,
    'rooms': property_rooms,
    'bathrooms': property_baths,
    'parkingLot': property_park,
    'privateArea': property_private_area,
    'constructionArea': property_const_area,
    'squareMetersPrice': json_finca_raiz['PriceM2'],
    'stratum': property_stratum,
    'status': property_feature_status,
    'antiquity': property_antiquity,
    'floors': property_floor,
    'weather': property_weather,
    'admonPrice': json_finca_raiz['AdministrationPrice'],
    'apartmentType': property_apartment_type
}
property_features.append(features_object)

# Extract Description
description = soup.find('div', class_='description')
property_code = int(description.find('span').find('b').get_text())
property_description = description.find('p').text

# Hidden Extra
extra = soup.find('div', id='DivEstrasHidden')
extras = []
for h4 in extra.find_all('h4'):
    extras.append(h4.text)

property_interior_features = []
property_exterior_features = []
property_sector_features = []

for extra in extras:
    if(extra.find('Caracteristicas Interiores') == 0): # Extract Interior Features
        interior_features = soup.find('ul', id='tblInitialInteriores')
        property_interior_features = [li.text for li in interior_features.find_all('li')]
    elif(extra.find('Caracteristicas Exteriores') == 0): # Extract Exterior Features
        exterior_features = soup.find('ul', id='tblInitialExteriores')
        property_exterior_features = [li.text for buy_new_property_dictli in exterior_features.find_all('li')]
    elif(extra.find('Caracteristicas del Sector') == 0): # Extract Sector Features
        sector_features = soup.find('ul', id='tblInitialdelSector')
        property_sector_features = [li.text for li in sector_features.find_all('li')]

## Property Scraping (New Properties)

In [565]:
paginator_url = 'https://www.fincaraiz.com.co/palermo/medellin/proyecto-nuevo-det-2393445.aspx'
page = requests.get(paginator_url)
soup = BeautifulSoup(page.content, 'html.parser')

# Extract Header
detail_header = soup.find('div', class_='row detailheader')

detail_title = detail_header.find('div', class_='title')
property_name_project = detail_title.find('div', class_='box').find('h1').text
property_address_project = detail_title.find('span', class_='address').text
property_neighborhood = detail_title.find('span', class_='font-light').text.split('-')
property_neighborhood = property_place[0][0:-1]

detail_price = detail_header.find('div', class_='price')
property_min_price = ''
property_max_price = ''
array_prices = []
for price in detail_price.find_all('label'):
    array_prices.append(price.text)
property_min_price = array_prices[0].split()
property_min_price = " ".join(property_min_price)
property_max_price = array_prices[1].split()
property_max_price = " ".join(property_max_price)

property_price = []
price_object = {
    'minPrice': property_min_price,
    'maxPrice': property_max_price
}
property_price.append(price_object)

# Extract Builder Company
contact_form = soup.find('div', class_='contact-form')
builder_company = contact_form.find('input', id='hd_UserName').attrs.get('value')

# Extract Small Features
small_features = soup.find('div', class_='features clearfix')
property_sqrt_meters = small_features.find('span', class_='advertSurface').get_text().split()
property_sqrt_meters = " ".join(property_sqrt_meters)[0:-3].replace(",", ".")
property_rooms = small_features.find('span', class_='advertRooms').get_text().split()
property_rooms = int(" ".join(property_rooms)[-1])
property_baths = small_features.find('span', class_='advertBaths').get_text().split()
property_baths = int(" ".join(property_baths)[-1])
property_park = small_features.find('span', class_='advertGarages').get_text().split()
property_park = property_park[-1]
if(property_park == 'especificar'):
    property_park = int('0')
else:
    property_park = int(property_park)

# Estract Big Features
big_features = soup.find('ul', class_='boxcube')
property_private_area = ''
property_const_area = ''
property_sqrt_meters_price = ''
property_stratum = ''
property_feature_status = ''
property_antiquity = ''
property_floor = ''
property_sector = ''
property_weather = ''
property_admon = ''
for li in big_features.find_all('li'):
    value = li.get_text().split()
    value = " ".join(value)
    if(value.find('Área privada') == 0):
        property_private_area = value.replace('Área privada: ', '')[0:-3]
    elif(value.find('Área Const') == 0):
        property_const_area = value.replace('Área Const.: ', '')[0:-3]
    elif(value.find('Precio m²') == 0):
        property_sqrt_meters_price = value.replace('Precio m²: ', '')[0:-3]
    elif(value.find('Estrato') == 0):
        property_stratum = int(value.replace('Estrato: ', '')[0])
    elif(value.find('Estado') == 0):
        property_feature_status = value.replace('Estado: ', '')
    elif(value.find('Antigüedad') == 0):
        property_antiquity = value.replace('Antigüedad: ', '')
    elif(value.find('Piso No') == 0):
        property_floor = value.replace('Piso No: ', '')[0:-1]
    elif(value.find('Sector') == 0):
        property_sector = value.replace('Sector: ', '')
    elif(value.find('Clima') == 0):
        property_weather = value.replace('Clima: ', '')
    elif(value.find('Admón:') == 0):
        property_admon = value.replace('Admón: ', '').split()[0]
    elif(value.find('Tipo de Apartamento') == 0):
        property_apartment_type = value.replace('Tipo de Apartamento: ', '')
        
property_features = []
features_object = {
    'squareMeters': property_sqrt_meters,
    'rooms': property_rooms,
    'bathrooms': property_baths,
    'parkingLot': property_park,
    'privateArea': property_private_area,
    'constructionArea': property_const_area,
    'squareMetersPrice': json_finca_raiz['PriceM2'],
    'stratum': property_stratum,
    'status': property_feature_status,
    'antiquity': property_antiquity,
    'floors': property_floor,
    'weather': property_weather,
    'admonPrice': json_finca_raiz['AdministrationPrice'],
    'apartmentType': property_apartment_type
}
property_features.append(features_object)

# Extract Description
description = soup.find('div', class_='description')
property_code = int(description.find('span').find('b').get_text())
property_description = description.find('p').text

# Hidden Extra
extra = soup.find('div', id='DivEstrasHidden')
extras = []
for h4 in extra.find_all('h4'):
    extras.append(h4.text)

property_interior_features = []
property_exterior_features = []
property_sector_features = []

for extra in extras:
    if(extra.find('Caracteristicas Interiores') == 0): # Extract Interior Features
        interior_features = soup.find('ul', id='tblInitialInteriores')
        property_interior_features = [li.text for li in interior_features.find_all('li')]
    elif(extra.find('Caracteristicas Exteriores') == 0): # Extract Exterior Features
        exterior_features = soup.find('ul', id='tblInitialExteriores')
        property_exterior_features = [li.text for li in exterior_features.find_all('li')]
    elif(extra.find('Caracteristicas del Sector') == 0): # Extract Sector Features
        sector_features = soup.find('ul', id='tblInitialdelSector')
        property_sector_features = [li.text for li in sector_features.find_all('li')]

# Extract Offers Types
offers_type = soup.find('div', id='typology')
table = offers_type.find('table')
array_offers_type = []
for tr in table.find_all('tr'):
    property = tr.find_all('td')[0].text.split()
    property = " ".join(property)
    offer_type = tr.find_all('td')[1].text.split()
    offer_type = " ".join(offer_type)
    area = tr.find_all('td')[2].text.split()
    area = " ".join(area)[0:-3].replace(",", ".")
    private_area = tr.find_all('td')[3].text.split()
    private_area = " ".join(private_area)[0:-3].replace(",", ".")
    rooms = tr.find_all('td')[4].text.split()
    rooms = " ".join(rooms)
    baths = tr.find_all('td')[5].text.split()
    baths = " ".join(baths)
    price = tr.find_all('td')[6].text.split()
    price = " ".join(price)[2:]
    
    table_object = {
        'property': property,
        'offerType': offer_type,
        'area': area,
        'privateArea': private_area,
        'rooms': rooms,
        'bathrooms': baths,
        'price': price
    }
    array_offers_type.append(table_object)

## Create JSON

In [566]:
import json

# Old Properties to Buy JSON
old_property_dict = {
    'code': property_code,
    'status': json_finca_raiz["TransactionType"],
    'use': 'Usado',
    'propertyAgency': json_finca_raiz["ClientName"],
    'price': json_finca_raiz["Price"],
    'country': 'Colombia',
    'department': 'Antioquia',
    'city': 'Medellin',
    'sector': property_sector,
    'neighborhood': json_finca_raiz["Neighborhood"],
    'location': property_array_location,
    'description': json_finca_raiz["Description"],
    'features': property_features,
    'interiorFeatures': property_interior_features,
    'exteriorFeatures': property_exterior_features,
    'sectorFeatures': property_sector_features
}

# New Properties to Buy JSON
new_property_dict = {
    'code': property_code,
    'nameProject': property_name_project,
    'builderCompany': json_finca_raiz["ClientName"],
    'status': json_finca_raiz["TransactionType"],
    'use': 'Nuevo',
    'price': property_price,
    'country': 'Colombia',
    'department': 'Antioquia',
    'city': 'Medellin',
    'sector': property_sector,
    'neighborhood': json_finca_raiz["Neighborhood"],
    'location': property_array_location,
    'description': json_finca_raiz["Description"],
    'features': property_features,
    'interiorFeatures': property_interior_features,
    'exteriorFeatures': property_exterior_features,
    'sectorFeatures': property_sector_features,
    'offersType': array_offers_type[1:]
}

print(json.dumps(old_property_dict, indent=4))

{
    "code": 2393445,
    "status": "Venta",
    "use": "Usado",
    "propertyAgency": "Constructora Editec",
    "price": "292633175",
    "country": "Colombia",
    "department": "Antioquia",
    "city": "Medellin",
    "sector": "Loma del Indio",
    "neighborhood": "Loma del Indio",
    "location": [
        {
            "latitude": "0",
            "longitude": "0"
        }
    ],
    "description": "PALERMO, donde las cosas importantes se encuentran muy cerca. Proyecto de apartamentos muy exclusivos, contiguo a la Iglesia SAN JOSE DE NAZARETH. Ac\u00e1 encontrar\u00e1 todo a la mano porque en nuestro d\u00eda a d\u00eda necesitamos de bancos, restaurantes, supermercados, estaciones de gasolina que nos hacen mas f\u00e1cil la vida. En el \u00faltimo piso de la torre tendremos un agradable Sky Club con jacuzzi para 10 personas, gimnasio, sal\u00f3n social y una espectacular vista..",
    "features": [
        {
            "squareMeters": "58.79 a 101.16",
            "rooms": 1